## Gathering Data (EDIT)


Publically available data containing movie genres, rating, and year of release has already been collected by various reputable sources. We chose to investigate the IMDb Datasets (https://www.imdb.com/interfaces/). These datasets have movie data going back all the way to               . The datasets also contain a wide range of potentially interesting data including: if it's an Adult movie, the production year, other(potentially foriegn) titles to that movie, names of directors, names of writers, etc. Another big positive to working with these datasets is that they are completely free to the public.  


Meanwhile with other movie datasets, while they are just as extensive as IMDb, they would require a payment to get the complete dataset and not just a sample. For instance, another database we were considering using was The Numbers (https://www.the-numbers.com/box-office-records/domestic/all-movies/cumulative/all-time/2501). However, we realised to get the data we required we would have to pay $475. 


## Data Processing


There times when the data you recieve from the databases are not in the correct format. Or, in a format that is difficult for the system to comprehend/handle. 

As mentioned above, one positive to IMDb Datasaets is that we would have a large dataset to analyze and explore. However, it is also a negative. Because it is so large (a total of 710 MB!!), it is impossible for most laptops/computers to process the data all at once. As a result, it must be condensed to only contain data we think may be relavent in future analysis and must be proccessed in chunks. 

NOTE: This is a common problem in data science and would most likely need to be addressed when you do data science analysis in the future.


In [16]:
import pandas as pd

#dictates the number of rows each data frame should have
chunksize = 100000

# the list that contains all the dataframes
list_of_dataframes = []

#Seperates title.basics.tsv into chunks with 100000 rows with the header of row 0.
#The columns of the dataframes would be tconst, originalTitle, startYear (publication year), 
#and genres data. each chunck is added to list_of_dataframes
for df in pd.read_csv("title.basics.tsv", sep='\t', chunksize=chunksize, 
                       header=0, usecols=['tconst', 'primaryTitle', 'genres', 'startYear']):
      list_of_dataframes.append(df)
        
#concats all the chunck dataframes to make one combined        
data1 = pd.concat(list_of_dataframes)


/Users/aaddeh/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [30]:
data1.head()

,tconst,primaryTitle,startYear,genres
0,tt0000001,Carmencita,1894,"Documentary,Short"
1,tt0000002,Le clown et ses chiens,1892,"Animation,Short"
2,tt0000003,Pauvre Pierrot,1892,"Animation,Comedy,Romance"
3,tt0000004,Un bon bock,1892,"Animation,Short"
4,tt0000005,Blacksmith Scene,1893,"Comedy,Short"


Each movie instance/row has the following features/attributes: 
* tconst  - IMDb's alphanumeric unique identifier of the movie title
* primaryTitle  – the more popular movie title / the movietitle used by the filmmakers on promotional     materials at the point of release
* startYear  – represents the release year of a movie title
* genres  – includes up to three genres associated with the title

All of them are object types.





In [32]:
#makes a dataframe from the data in the title.ratings.tsv database
#averageRating is the average IMDb rating , numVotes is the number of IMDd
#votes the movie recieved
data2= pd.read_csv("title.ratings.tsv", sep='\t')
data2.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1718
1,tt0000002,6.0,211
2,tt0000003,6.5,1480
3,tt0000004,6.1,124
4,tt0000005,6.2,2283


Each movie instance/row has the following features/attributes: 
* tconst - IMDb's alphanumeric unique identifier of the movie title (same as dataframe above)
* averageRating  – weighted average of all the individual user ratings
* numVotes  - number of votes the title has received

All of them are object types.


#### Processing unavailable data in the Dataframe

It is very important the data within the dataframe that is unavailable is categorized properly and uniformly.

There is no uniform rule on how different databases will represent missing data. Some databases put '-', others write 'NONE', or an empty string. There is an infinite amount of possibilities that databases could use. However, what is important from the data science perspective is how will Pandas interpret those values and having a concesus within the community. 

Inorder to Pandas (and the community) to understand that a value is missing, that cell must have one of these values:

* numpy.NaN (numpy): A number  value is not available in a number column
* numpy.nan (numpy): Same as the above (either one could be use for a missing number)
* pd.NaT (pandas):A time value in a time column is not available
* pd.NA (pandas): A general type of value is Not Available
* None (python): similar to pd.NA

Note: Data has an underlying meaning. If something was meant to be number but turned out not to be, that could be potentially different from it not being available. 

According to the IMBd website, their databases use ‘\N’ to denote that a particular field is missing or null for that title/name. These must be converted to an appropriate value from the list above. 


In [45]:
import numpy 

#replaces all instance of missing values in the data1 dataframe as replace it with a pd Null or numpy Null value
data1["tconst"].replace({"\\N": pd.NA}, inplace=True)
data1["primaryTitle"].replace({"\\N": pd.NA}, inplace=True)
data1["genres"].replace({"\\N": pd.NA}, inplace=True)
data1["startYear"].replace({"\\N": pd.NaT}, inplace=True) # replaces unavaiable year data with pandas NaT

#replaces all instance of missing values in the data2 dataframe as replace it with a pd Null or numpy Null value
data2["tconst"].replace({"\\N": pd.NA}, inplace=True)
data2["averageRating"].replace({"\\N":numpy.NaN}, inplace=True)
data2["numVotes"].replace({"\\N":numpy.NaN}, inplace=True)


In [52]:
data1

,tconst,primaryTitle,startYear,genres
0,tt0000001,Carmencita,1894,"Documentary,Short"
1,tt0000002,Le clown et ses chiens,1892,"Animation,Short"
2,tt0000003,Pauvre Pierrot,1892,"Animation,Comedy,Romance"
3,tt0000004,Un bon bock,1892,"Animation,Short"
4,tt0000005,Blacksmith Scene,1893,"Comedy,Short"
...,...,...,...,...
8070583,tt9916848,Episode #3.17,2010,"Action,Drama,Family"
8070584,tt9916850,Episode #3.19,2010,"Action,Drama,Family"
8070585,tt9916852,Episode #3.20,2010,"Action,Drama,Family"
8070586,tt9916856,The Wind,2015,Short
